In [1]:
import torch
import torch.nn as nn
import torch.optim as optimiser
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as Func
import pandas as pd
import time

In [2]:
torch.cuda.is_available()

True

In [4]:
# Defining the transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),# flipping the image/mirroring it
    transforms.RandomCrop(32, padding=4), # cropping randomly
    transforms.ToTensor(), # convert image to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalises to mean:0 and std:1
])


# Loading the CIFAR-10 dataset / Getting cifar-10 (test/train) and preprocessing using transform
batchsize = 32
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batchsize, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=batchsize, shuffle=False, num_workers=2)

class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()

        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn_1 = nn.BatchNorm2d(12)
        self.conv_2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn_2 = nn.BatchNorm2d(24)
        self.pool_1 = nn.MaxPool2d(2,2)
        self.conv_4 = nn.Conv2d(in_channels= 24, out_channels=48, kernel_size=5, stride=1, padding=1)
        self.bn_4 = nn.BatchNorm2d(48)
        self.pool_2 = nn.MaxPool2d(2,2)
        self.conv_5 = nn.Conv2d(in_channels= 48, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn_5 = nn.BatchNorm2d(24)
        self.conv_6 = nn.Conv2d(in_channels= 24, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn_6 = nn.BatchNorm2d(12)

        self.fL_1 = nn.Linear(12*10*10, 10)


    def forward(self, x):

        output_m = Func.relu(self.bn_1(self.conv_1(x)))
        output_m = Func.relu(self.bn_2(self.conv_2(output_m)))
        output_m = self.pool_1(output_m)
        output_m = Func.relu(self.bn_4(self.conv_4(output_m)))
        output_m = Func.relu(self.bn_5(self.conv_5(output_m)))

        output_m = Func.relu(self.bn_6(self.conv_6(output_m)))
        output_m = output_m.view(-1, 12*10*10)
        #output_m = output_m.view(-1, 24*10*10)

        output_m = self.fL_1(output_m)

        return output_m

basic_cnn_model = BasicCNN()

loss_func = nn.CrossEntropyLoss()
optimizer = optimiser.Adam(basic_cnn_model.parameters(), lr=0.01)

def training_model(model, optimizer, epochs=25):
    training_accuracy_history = []
    validation_accuracy_history = []
    training_loss_history = []
    validation_loss_history = []

    device_test = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device_test)

    starting_time = time.time()

    for ep in range(epochs):
        model.train()
        run_loss = 0.0
        correct_train = 0
        total_train = 0

        for data in train_loader:
            input_train, label_train = data
            input_train, label_train = input_train.to(device_test), label_train.to(device_test)
            optimizer.zero_grad()

            output_train = model(input_train)
            loss_train = loss_func(output_train, label_train)
            loss_train.backward()
            optimizer.step()

            run_loss =  run_loss + loss_train.item()
            torch_val, predicted_train = torch.max(output_train.data, 1)
            total_train = total_train + label_train.size(0)
            correct_train = correct_train + (predicted_train == label_train).sum().item()

        training_loss = run_loss / len(train_loader)
        training_accuracy = 100 * correct_train / total_train
        training_loss_history.append(training_loss)
        training_accuracy_history.append(training_accuracy)

        # Doing the Validating
        model.eval()
        vali_correct = 0
        vali_total = 0
        validation_loss = 0.0

        with torch.no_grad():
            for data in test_loader:
                vali_input, vali_label = data
                vali_input, vali_label = vali_input.to(device_test), vali_label.to(device_test)
                vali_output = model(vali_input)
                vali_loss = loss_func(vali_output, vali_label)
                validation_loss = validation_loss + vali_loss.item()
                torch_val, predicted_vali = torch.max(vali_output.data, 1)
                vali_total = vali_total + vali_label.size(0)
                vali_correct = vali_correct +  (predicted_vali == vali_label).sum().item()

        validation_loss = validation_loss / len(test_loader)
        vali_accuracy = 100 * vali_correct / vali_total
        validation_loss_history.append(validation_loss)
        validation_accuracy_history.append(vali_accuracy)

        print(f'Epoch Count [{ep + 1}/{epochs}], '
              f'Training Loss: {training_loss:.4f}, '
              f'Training Accuracy: {training_accuracy:.2f}%, '
              f'Validation Loss: {validation_loss:.4f}, '
              f'Validation Accuracy: {vali_accuracy:.2f}%')


    ending_time = time.time()  # Record the end time
    total_elapsed_time = ending_time - starting_time  # Calculate elapsed time

    print(f'The training time is : {total_elapsed_time:.2f} seconds')  # Print the training time

    return training_accuracy_history, training_loss_history, validation_accuracy_history, validation_loss_history

# Training the basic CNN model
training_accuracy_history, training_loss_history, validation_accuracy_history, validation_loss_history = training_model(basic_cnn_model, optimizer)




Files already downloaded and verified
Files already downloaded and verified
Epoch Count [1/25], Training Loss: 1.7587, Training Accuracy: 35.26%, Validation Loss: 1.4995, Validation Accuracy: 44.59%
Epoch Count [2/25], Training Loss: 1.3899, Training Accuracy: 49.71%, Validation Loss: 1.2512, Validation Accuracy: 54.43%
Epoch Count [3/25], Training Loss: 1.1794, Training Accuracy: 58.29%, Validation Loss: 1.1752, Validation Accuracy: 59.44%
Epoch Count [4/25], Training Loss: 1.0637, Training Accuracy: 62.77%, Validation Loss: 1.0100, Validation Accuracy: 64.37%
Epoch Count [5/25], Training Loss: 0.9884, Training Accuracy: 65.29%, Validation Loss: 1.0118, Validation Accuracy: 64.67%
Epoch Count [6/25], Training Loss: 0.9468, Training Accuracy: 66.71%, Validation Loss: 1.0399, Validation Accuracy: 64.05%
Epoch Count [7/25], Training Loss: 0.9045, Training Accuracy: 68.38%, Validation Loss: 0.9268, Validation Accuracy: 67.16%
Epoch Count [8/25], Training Loss: 0.8750, Training Accuracy: 6